# Running GMM

### Imorting packages and data

In [1]:
#%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import six
import tensorflow as tf
import os
import pandas as pd

from edward.models import Categorical

#from Models.GMM.Save import simple_save

from edward.models import (
    Categorical, Dirichlet, Empirical, InverseGamma,
    MultivariateNormalDiag, Normal, ParamMixture, RandomVariable, Bernoulli)
from tensorflow.contrib.distributions import Distribution

plt.style.use('ggplot')


###########
#For saving models:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.framework import ops
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
#from tensorflow.python.util.tf_export import tf_export

#Reading data:
subset= pd.read_csv('\user_rec.csv', sep=",", header=0)
x_trainn=subset.values                                                  #Redefine x_train into an array instead of dataframe

#Standardizing:
from sklearn import preprocessing
X= preprocessing.scale(x_trainn)


#Defining test & train:
x_train= X[0:4984,]                                                  #Defining training data
x_test= X[4984:,]                                                   #Defining test data
N_testt= len(x_test)                                                    #Defining the length of the test-set


N = len(x_train)  # number of data points                               #Setting parameters - N is defined from the number of rows
K = 30  # number of components                                           #Setting parameters - number of clusters
D = x_train.shape[1]  # dimensionality of data                           #Setting parameters - dimension of data
ed.set_seed(42)


C:\Users\anders\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


### Definerer modellen

In [ ]:
#Model:
pi = Dirichlet(tf.ones(K))                                              
                                                                        
mu = Normal(tf.zeros(D), tf.ones(D), sample_shape=K)                    
sigmasq = InverseGamma(tf.ones(D), tf.ones(D), sample_shape=K)          
                                                                        
x = ParamMixture(pi, {'loc': mu, 'scale_diag': tf.sqrt(sigmasq)},       
                 MultivariateNormalDiag,                                
                 sample_shape=N)
z = x.cat                                                               


#Inference:                                                             
T = 800                                                                
qpi = Empirical(                                                       
    tf.get_variable(                                                   
    "qpi/params", [T, K],                                              
    initializer=tf.constant_initializer(1.0 / K)))                     
qmu = Empirical(tf.get_variable(                                       
    "qmu/params", [T, K, D],
    initializer=tf.zeros_initializer()))
qsigmasq = Empirical(tf.get_variable(                                  
    "qsigmasq/params", [T, K, D],
    initializer=tf.ones_initializer()))
qz = Empirical(tf.get_variable(                                        
    "qz/params", [T, N],
    initializer=tf.zeros_initializer(),
    dtype=tf.int32))

#Running Gibbs Sampling:
inference = ed.Gibbs({pi: qpi, mu: qmu, sigmasq: qsigmasq, z: qz},     
                     data={x: x_train})                                 
inference.initialize()                                                  

sess = ed.get_session()
tf.global_variables_initializer().run()

t_ph = tf.placeholder(tf.int32,[])                                      
running_cluster_means = tf.reduce_mean(qmu.params[:t_ph], 0)            

for _ in range(inference.n_iter):
  info_dict = inference.update()
  inference.print_progress(info_dict)
  t = info_dict['t']
  if t % inference.n_print == 0:
    print("\nInferred cluster means:")
    print(sess.run(running_cluster_means, {t_ph: t - 1}))                

C:\Users\anders\Anaconda3\envs\tensorflow\lib\site-packages\edward\util\random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


  8/800 [  1%]                                ETA: 1803s | Acceptance Rate: 1.000
Inferred cluster means:
[[-1.02308476e+00  1.86625218e+00  1.48905373e+00  4.06379431e-01
   6.94599032e-01  4.05242369e-02  1.37181735e+00  2.53820688e-01
   1.25883257e+00]
 [-7.78559685e-01  5.22398651e-01  6.73868805e-02 -5.64553738e-01
  -3.03174496e-01 -9.47878137e-02 -9.09302384e-02 -1.28014937e-01
   9.22039673e-02]
 [-8.94235015e-01  4.53304082e-01  5.46624474e-02 -1.66956857e-01
   7.63227120e-02 -5.45887975e-03 -7.79626379e-03 -1.52375594e-01
   5.44275224e-01]
 [-7.54574955e-01 -3.62551585e-02 -1.15349458e-03 -7.13007301e-02
  -1.78202447e-02  1.12625316e-01 -2.06632577e-02  1.53725460e-01
  -1.37231797e-02]
 [-1.03083146e+00 -3.69775034e-02 -1.99058233e-03  1.31207061e+00
   1.24867363e-02  2.88690299e-01 -1.69581044e-02 -1.41825452e-01
   1.96834743e+00]
 [-2.11636275e-01  3.39262754e-01  1.54110879e-01 -4.26771790e-01
  -5.85541129e-01  5.46487942e-02  1.42242229e-02 -9.08525661e-02
   9.88

 24/800 [  3%]                                ETA: 2430s | Acceptance Rate: 1.000
Inferred cluster means:
[[-1.00006485e+00  8.21694672e-01  5.42975843e-01  2.72661328e-01
   4.77373898e-01  6.95851594e-02  9.35197055e-01  4.77299869e-01
   1.25131059e+00]
 [-4.30457741e-01  2.55337685e-01 -2.14215770e-01 -5.99069186e-02
  -1.37812555e-01 -8.99588764e-02 -1.27390072e-01  4.26508021e-03
   8.88647586e-02]
 [-9.94802237e-01  1.25725675e+00  2.12033585e-01 -2.98145473e-01
   2.22431228e-01  1.07572442e-02 -2.41588950e-02 -1.95648670e-01
   1.10044670e+00]
 [-5.45068681e-01 -2.93329377e-02  1.30493686e-01 -1.65292889e-01
   2.23921724e-02  5.95961273e-01 -6.68651192e-03  9.54012796e-02
  -1.87320724e-01]
 [-1.03097010e+00 -2.16359850e-02  2.08223328e-01  6.76236093e-01
  -1.46397706e-02  1.68188512e+00 -1.64256413e-02 -1.65198430e-01
   1.72277904e+00]
 [ 4.58080880e-02  2.56837040e-01  3.18288356e-01 -9.73595306e-02
   4.67298031e-02  6.40433282e-02 -3.59188095e-02 -1.11149170e-01
  -2.80

 40/800 [  5%] █                              ETA: 2549s | Acceptance Rate: 1.000
Inferred cluster means:
[[-1.00895143e+00  4.82129127e-01  3.90999734e-01  2.33694047e-01
   5.54912925e-01  7.91620165e-02  6.01182401e-01  3.09762657e-01
   1.24428809e+00]
 [-2.48989299e-01  2.06666946e-01 -3.32511067e-01  1.56895369e-02
   9.07384604e-02 -1.68074705e-02 -2.06411898e-01 -4.11949083e-02
  -1.38444975e-01]
 [-1.00813532e+00  1.53712010e+00  2.63290823e-01 -2.82970488e-01
   2.81886190e-01  1.81672703e-02 -2.69398931e-02 -2.03186676e-01
   1.18028653e+00]
 [-4.32740867e-01 -2.84404699e-02  1.90912694e-01 -1.31201074e-01
   3.61060537e-02  6.94158554e-01  3.27530521e-04  7.71734044e-02
  -1.32918864e-01]
 [-1.01768684e+00 -2.44032443e-02  3.28524888e-01  5.87041557e-01
  -4.54226742e-03  2.11797333e+00 -2.63741352e-02 -1.00230679e-01
   1.38184178e+00]
 [-2.32564528e-02  1.68181732e-01  2.38433614e-01 -1.29224509e-01
  -9.96946022e-02  6.87629282e-02  1.35549521e-02  8.28474090e-02
  -1.83

 56/800 [  7%] ██                             ETA: 2552s | Acceptance Rate: 1.000
Inferred cluster means:
[[-9.82910752e-01  4.41022158e-01  3.81490469e-01  3.35062325e-01
   4.53764468e-01  7.64478371e-02  3.74585241e-01  3.83846700e-01
   1.17145228e+00]
 [-1.72654450e-01  1.95261776e-01 -1.54798657e-01  3.80197577e-02
  -8.36614054e-03 -8.92361104e-02 -1.96552098e-01 -1.00170940e-01
  -2.55981665e-02]
 [-1.02198923e+00  1.65571332e+00  2.92488903e-01 -2.88693488e-01
   3.04512888e-01  2.28729993e-02 -3.07030845e-02 -2.01624066e-01
   1.22432554e+00]
 [-3.82923603e-01 -2.05395631e-02  2.22960427e-01 -7.15167895e-02
   3.13657895e-02  7.53504157e-01 -2.36754931e-04  5.70570268e-02
  -1.08455718e-01]
 [-9.74221408e-01 -4.59676906e-02  5.00128865e-01  4.97680724e-01
   4.34763916e-02  2.04776573e+00 -4.01377454e-02 -3.87777574e-02
   1.26219869e+00]
 [-5.21327071e-02  1.33039802e-01  1.72052801e-01 -1.95222767e-03
  -6.40915111e-02  2.51966212e-02  1.27868190e-01  1.94706485e-01
  -1.42

...

584/800 [ 73%] █████████████████████          ETA: 782s | Acceptance Rate: 1.000
Inferred cluster means:
[[-6.39717519e-01  1.46981567e-01  1.14747494e-01  3.46660793e-01
   7.33970329e-02  1.66959688e-01  1.44771338e-01  5.77472866e-01
   8.17977786e-01]
 [-2.39213146e-02  3.51829678e-02 -4.89509702e-02  3.20063229e-03
  -1.03768244e-01 -6.61854446e-03 -8.96556601e-02 -3.22896838e-02
   3.68790291e-02]
 [-1.04151726e+00  1.96916389e+00  3.81263733e-01 -2.42435127e-01
   4.17628855e-01  2.81412974e-02 -3.04602068e-02 -2.02522337e-01
   1.30435967e+00]
 [-4.84564811e-01 -2.44609658e-02  3.74938756e-01  6.38402104e-02
   9.88302007e-03  1.66121125e+00  1.09728845e-02 -1.30088273e-02
   2.32807249e-01]
 [-1.49717137e-01 -1.81910414e-02  1.91978127e-01  4.33161370e-02
   1.12359680e-01  2.49645755e-01  4.12490554e-02 -7.92895705e-02
   1.85620725e-01]
 [ 6.12931624e-02  2.97561400e-02  1.20864853e-01  2.14558169e-02
  -4.43478487e-02 -1.86454840e-02  3.06123756e-02  3.64146829e-02
   2.531

600/800 [ 75%] ██████████████████████         ETA: 724s | Acceptance Rate: 1.000
Inferred cluster means:
[[-6.3572341e-01  1.4217269e-01  1.2440966e-01  3.4734479e-01
   6.8247616e-02  1.7420696e-01  1.4842337e-01  5.7944715e-01
   8.3073670e-01]
 [-3.3605102e-02  3.4274016e-02 -3.8746551e-02  3.3369190e-03
  -1.0171134e-01 -4.1146162e-03 -8.8179536e-02 -3.3303153e-02
   2.9778896e-02]
 [-1.0405082e+00  1.9730514e+00  3.8187078e-01 -2.4112420e-01
   4.1965497e-01  2.7890584e-02 -3.0779019e-02 -2.0256372e-01
   1.3027514e+00]
 [-4.8649910e-01 -2.4838353e-02  3.7679392e-01  6.4030543e-02
   9.5209610e-03  1.6703539e+00  1.1137970e-02 -1.2623693e-02
   2.3357013e-01]
 [-1.5295921e-01 -1.4413470e-02  1.9229394e-01  3.9945405e-02
   1.0267340e-01  2.4045101e-01  3.3654097e-02 -9.1518797e-02
   1.8730639e-01]
 [ 5.6258112e-02  3.3673197e-02  1.1597012e-01  1.1150746e-02
  -5.4175891e-02 -1.7123500e-02  3.8264807e-02  2.8182037e-02
   2.8382570e-02]
 [-5.0940268e-02  6.2646702e-02  2.7342808e

616/800 [ 77%] ███████████████████████        ETA: 666s | Acceptance Rate: 1.000
Inferred cluster means:
[[-6.31105959e-01  1.36735290e-01  1.18819818e-01  3.54689837e-01
   6.25895858e-02  1.76112458e-01  1.40746787e-01  5.66336751e-01
   8.27367783e-01]
 [-2.72658598e-02  3.16338129e-02 -3.70449759e-02  1.10917473e-02
  -9.28468034e-02 -4.87859873e-03 -9.01138708e-02 -3.31163965e-02
   1.74696539e-02]
 [-1.03890610e+00  1.97584474e+00  3.81716371e-01 -2.39905328e-01
   4.21550125e-01  2.79357824e-02 -3.05037610e-02 -2.02673480e-01
   1.30319035e+00]
 [-4.87608969e-01 -2.50388999e-02  3.78298610e-01  6.35317788e-02
   9.43577476e-03  1.68376470e+00  1.11836381e-02 -1.33881513e-02
   2.34518558e-01]
 [-1.43693104e-01 -1.35676032e-02  1.95329234e-01  4.42326106e-02
   1.02209955e-01  2.30635062e-01  3.35862748e-02 -6.87491000e-02
   1.79835573e-01]
 [ 6.05035312e-02  4.61365692e-02  1.15814082e-01  1.23442747e-02
  -4.54441458e-02 -1.39570907e-02  4.56538238e-02  3.04013174e-02
   2.862

632/800 [ 79%] ███████████████████████        ETA: 608s | Acceptance Rate: 1.000
Inferred cluster means:
[[-6.27122819e-01  1.34614766e-01  1.14739843e-01  3.60988170e-01
   6.01018630e-02  1.72679365e-01  1.41495034e-01  5.67492664e-01
   8.42458010e-01]
 [-3.78032848e-02  3.40759307e-02 -3.78170423e-02  8.42584763e-03
  -9.96796265e-02  4.25505964e-03 -7.97564685e-02 -3.73961441e-02
   2.84131989e-02]
 [-1.03803873e+00  1.97595870e+00  3.82907838e-01 -2.39482269e-01
   4.22146559e-01  2.76938975e-02 -3.12885121e-02 -2.02835694e-01
   1.30361617e+00]
 [-4.87903863e-01 -2.55514588e-02  3.81627560e-01  6.40685037e-02
   9.22050234e-03  1.68521881e+00  1.14466231e-02 -1.42275924e-02
   2.32968584e-01]
 [-1.36153653e-01 -1.75254531e-02  1.88986033e-01  3.81086878e-02
   8.58082175e-02  2.24537745e-01  3.41516621e-02 -7.45909736e-02
   1.78819597e-01]
 [ 6.28989562e-02  5.32567091e-02  1.14161365e-01  2.22949442e-02
  -4.80947308e-02 -2.72931773e-02  4.11592014e-02  2.90212557e-02
   3.753

648/800 [ 81%] ████████████████████████       ETA: 550s | Acceptance Rate: 1.000
Inferred cluster means:
[[-6.22658074e-01  1.28727078e-01  1.11823350e-01  3.64053965e-01
   5.39719351e-02  1.76624015e-01  1.48771808e-01  5.51393032e-01
   8.44877899e-01]
 [-3.00502311e-02  3.29002738e-02 -2.66644396e-02  1.99622810e-02
  -9.64658558e-02  1.40824681e-02 -5.97487949e-02 -3.48466821e-02
   2.02053301e-02]
 [-1.03778648e+00  1.97713435e+00  3.84603977e-01 -2.36895949e-01
   4.24388230e-01  2.79038530e-02 -3.08029428e-02 -2.02907279e-01
   1.30541468e+00]
 [-4.87633258e-01 -2.53479760e-02  3.83803695e-01  6.49854764e-02
   8.80200043e-03  1.68951106e+00  1.14522856e-02 -1.46769471e-02
   2.34477431e-01]
 [-1.39256880e-01 -2.45426316e-02  1.91242442e-01  2.88332775e-02
   8.24976936e-02  2.26802155e-01  3.82617973e-02 -8.02923739e-02
   1.82428762e-01]
 [ 5.45416810e-02  5.13801388e-02  1.17151298e-01  2.91403607e-02
  -5.46265990e-02 -1.81397516e-02  3.19018960e-02  3.78608629e-02
   4.166

## Criticism

###  Train: Sampling from posterior

In [ ]:
#Criticism:
# Calculate likelihood for each data point and cluster assignment,
# averaged over many posterior samples. `x_post` has shape (N, 100, K, D).
pi_sample = qpi.sample(100)
mu_sample = qmu.sample(100)                                                 
sigmasq_sample = qsigmasq.sample(100)                                       
x_post = Normal(loc=tf.ones([N, 1, 1, 1]) * mu_sample,                      
                scale=tf.ones([N, 1, 1, 1]) * tf.sqrt(sigmasq_sample))      
                                                                            
                                                                            
x_broadcasted = tf.tile(tf.reshape(x_train, [N, 1, 1, D]), [1, 100, K, 1])  
x_broadcasted = tf.cast(x_broadcasted, tf.float32)                          

### Train: Running log_liks

In [ ]:
#Log_liks:
log_liks = x_post.log_prob(x_broadcasted)                                   
log_liks = tf.reduce_sum(log_liks, 3)                                       
log_liks = tf.add(log_liks,tf.log(pi_sample))                               
log_liks = tf.subtract(tf.reduce_logsumexp(log_liks, 1),tf.log(100.0))      
log_dens=tf.reduce_logsumexp(log_liks,1)                                    

###  Train:  Samlet log_liks & cluster assignment

In [ ]:
tf.reduce_sum(log_dens).eval()                                

In [ ]:
#cluster with the highest likelihood for each data point assigned cluster value.
clusters = tf.argmax(log_liks, 1).eval()                    

### Test: Sampling

In [ ]:
#Prediction on test data
x_broadcasted = tf.tile(tf.reshape(x_test, [N, 1, 1, D]), [1, 100, K, 1]) 
x_broadcasted = tf.cast(x_broadcasted, tf.float32)                        

### Test: Log_liks

In [ ]:
#Log_liks:
log_liks = x_post.log_prob(x_broadcasted)                                   
log_liks = tf.reduce_sum(log_liks, 3)                                       
log_liks = tf.add(log_liks,tf.log(pi_sample))                               
log_liks = tf.subtract(tf.reduce_logsumexp(log_liks, 1),tf.log(100.0))      
log_dens=tf.reduce_logsumexp(log_liks,1)                                    

### Test: Samplet log_liks og cluster

In [ ]:
tf.reduce_sum(log_dens).eval()   

In [ ]:
#Cluster assignments:
clusters = tf.argmax(log_liks, 1).eval()     